## Running experiments with local resources

In this example, all the compute and storage are local, and we only use Azure to log the metrics from our experiments

In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print(f"Using {ws.name} with version {azureml.core.VERSION}")

Using azml-sdk with version 1.30.0


In [3]:
from azureml.core import Experiment
import pandas as pd

experiment = Experiment(workspace=ws, name="diabetes-local")
run = experiment.start_logging()
print(f"Starting experiment: {experiment.name}")

Starting experiment: diabetes-local
